# Parsing

## Guide to code:

We must underline that to find each element we take advantage of the Wikipedia website structure: we have seen that this structure is more or less always the same. We add if conditions to overcome the changes in the structure that didn't allow us to find the data. For what we have just explained, we warmly recomend to use wikipedia API to get intro and plot and for the info_box it would be better to find a solution that would be able to find data in any circumstances.

In [1]:
import requests as req
from bs4 import BeautifulSoup
import random
import time
import os
import codecs
import re
import json
import numpy as np
import collections
import csv
import traceback

In [2]:
# Let's create a function to get the info box:

## Guide to code:

First two ifs statements need to avoid two kind of pages that we are not interested in (in order): pages that don't exist anymore (we return -1, because later it will take less time to be found than a string) and pages that should bring the user to other websites (we put "NA", because in this case we want to add at least the title).

We have seen that the intro is always placed after the infobox table, so we start to import the paragraphs of the intro from there. However, if there isn't the infobox the intro starts after the line that defines the main section of the page, in this case we should use nextChild function instead nextSibling because of the hirarchy of tags. We will loop till we find p tags (the tags for paragraphs), as soon as the tag changes we will stop the cycle and return the result.

In [3]:
def find_intro(site):

    # Deal with Page not FOund:
    
    if "Page not Found" is in site.text: # If the isn't the page we return -1 it takes less time to find it later
        return -1
    
    # Deal with Disanbiguation page:
    
    disanbiguation = site.find("a", {"title": "Category:Disambiguation pages"})
    if disanbiguation: # Disanbiguation page = a page that sends you to other pages (we decide to skip them, we add only the title)
        return "NA"
    
    # Leet's start to find intro:
    
    intro_lst = [] # A list which will contain each paragraph
    par_before_intro = site.find("table", {"class": "infobox vevent"}) # It is the element before the paragrphs we need
    
    if par_before_intro:
        el = par_before_intro.find_next_sibling()  # The element changes everytime the cycle ends (at the beginning we start from the line after table tag
    else:  # If there isn't the info box, we can find the intro right after the following tag (but it is a child of this)
        el = site.find("div", {"class": "mw-parser-output"}).findChild()
        
    while True:
        
        try:
            el_name = el.name
        except:
            el_name = "" # In case we don't get the tag 

        if el_name != "p": # We stop when we found the first tag different from p
            intro = "\n".join(intro_lst) # So we create a string with the whole text
            break # And stop the cycle
        else:
            intro_lst.append(el.text) # We append that part as text in our list
            el = el.find_next_sibling() # And we switch to the next element
        
    return intro

In [4]:
# Let's create a function to get the plot:

## Guide to code:

As for the intro, the first two lines are made to deal with pages that we don't need.

We have noticed that the paragphs of the plot start after the the title plot (contained in h2, whith the name write in span tag in h2), this title cna slightly changes, so we have written different if stataments for each possible event. 

Notice that the list plot_par contains always only element, which is the one descibed above (for construction).
Furthermore, it can happen that thereisn't the plot, in that case we just return a NA.

As for intro, after that so we start to import the paragraphs from the next sibling of what we wrote above.  We will loop till we find p tags (the tags for paragraphs), as soon as the tag changes we will stop the cycle and return the result.

In [5]:
def find_plot(site):

    # Deal with page not found
    
    if "Page not Found" is in site.text: # If the isn't the page we return -1 it takes less time to find it later
        return -1
    
    # Deal with disanbiguation page:
    
    disanbiguation = site.find("a", {"title": "Category:Disambiguation pages"})
    if disanbiguation: # Disanbiguation page = a page that sends the user to other pages (we decide to skip them, we add only the title)
        return "NA"
    
    # Let's start to find the plot
    
    plot_lst = [] # A list which will contain each paragraph
    paragraphs = site.find_all("h2") # All elements in site that have the tag "h2"
    plot_par = [i for i in paragraphs if (i.find("span", {"id": "Plot"}) or i.find("span", {"id": "Plot_summary"})
                                          or i.find("span", {"id": "Premise"}) or i.find("span", {"id": "Premise"}))]
    
    if not plot_par: # If there isn't the plot paragraph
        return "NA"
    
    el = plot_par[0].find_next_sibling() # The element changes everytime the cycle ends (at the beginning we start from the line after h2 tag)
    while True:

        try:
            el_name = el.name
        except:
            el_name = "" # In case we don't get the tag 

        if el_name != "p": # When occurs the first tag different from p we stop the loop
            plot = "\n".join(plot_lst) # So we create a string with the whole text
            break # And stop the cycle
        else:
            plot_lst.append(el.text) # We append that part as text in our list
            el = el.find_next_sibling() # And we switch to the next element

    return plot    

In [6]:
# Let' create a function to get info box's data:

## Guide to code:

Whit this function we want get from the infobox only the informatio asked from the exercise, so we have created a dictionary which contains as key what the information is about and as values "NA", which will be replace with the data that we will find in the infobox.

Also in this case we deal with page not found and disanbiguation page, but this time we will return a dictionary and simply a number/string (instead the final function to produce file.tsv won't work).

After getting the infobox table, we get the object title from the tag "tr" and the value from tag "th". We have developed some if statements to add each element to correct key in the dictionary. For the Title, it seems that this is the only element that is a "values" = "NA" (but we cannot 100% that it work for every files).

In [7]:
def find_info(site):

# Let's create a dictionary whith the neeed information
    
    info_dict = {"Title": "NA",
                "Directed by": "NA",
                "Produced by": "NA",
                "Written by": "NA",
                "Starring": "NA",
                "Music by": "NA",
                "Release date": "NA",
                "Country": "NA",
                "Language": "NA",
                "Budget": "NA" 
                }
    
    # Deal with Page not found:
    
    if "Page not Found" is in site.text: # If the isn't the page we return -1 it takes less time to find it later
        for key, _ in info_dict.items():
            info_dict[key] = -1
        return info_dict
    
    # Deal with disanbiguation page:
    
    disanbiguation = site.find("a", {"title": "Category:Disambiguation pages"})
    if disanbiguation: # Disanbiguation page = a page that sends you to other pages (we decide to skip them, we add only the title)
        info_dict["Title"] = site.find("h1").text
        return info_dict
    
    # Let's loop:
    
    table = site.find('table', {"class" : "infobox vevent"}) # Let's get the table where the infobox is palced
    
    if not table: # It can happen that there isn't the infobox
        return info_dict
    
    rows = table.find_all("tr") # It represents the table records
                
    for el in rows:

        try:
            key = el.find("th").get_text(strip=True) # They are the Titles (Ex. "Prodced by")
        except:
            key = "NA" 

        try:
            values = el.find("td").get_text(strip=True) # The attributes of each title
        except:
            values = "NA"

        if values == "NA": # It seems title is the only one that has values == "NA"
            
            info_dict["Title"] = key
                    
        elif key == "Directed by":
            
            info_dict[key] = values
            
        elif key == "Produced by":
            
            info_dict[key] = values
                
        elif key == "Written by":
            
            info_dict[key] = values
        
        elif key == "Starring":
            
            info_dict[key] = values
       
        elif key == "Music by":
            
            info_dict[key] = values
        
        elif key == "Release date":
            
            info_dict[key] = values
        
        elif key == "Running time":
         
            info_dict[key] = values
        
        elif key == "Country":

            info_dict[key] = values
        
        elif key == "Language":

            info_dict[key] = values
        
        elif key == "Budget":

            info_dict[key] = values
                
    return info_dict

In [8]:
# Let's create a function to clean data from html symbols and from some errors that may lead to errors:

# Guide to code:

In this part we are going to solve all the errors that we could observe opening some files. To be cleare this is not needed for the inverted index, but it will help to have more precise result later.

For i.e. we have noticed that Wikipedia uses lables such as ("[n]") to refer to some external link, or sometimes after converting in text some html elements remains such as ("\n","\s", ...).

In addition sometimes dates are written in weired way, our aim is to get only the year, another example can be that many times name are all together (e.x. "MarioRossiLucaFalco" or "Mario RossiLuca Falco").

So we are going to deal with all these kind of errors that we have seen. Of course, these are not all the errors can be occur, so we cannot guarantee that all words will be clean.

In [13]:
def clean_all(site):
    
    raw_intro = find_intro(site)
    raw_plot = find_plot(site)
    raw_info = find_info(site)
    
    # Let's clean the text from any html symhols and to wikipedia lables ("[n]") for intro an dplot:
    
    if type(raw_intro) is int: # It is page noy found, so we can skip it
        intro = raw_intro
        plot = raw_plot
        info_dict = raw_info
    
    else:
    
        intro_not_clean = re.sub('(?<=\w|\.|\s)((\\\'s)|(\\n){,10}|(\\\'))', "", raw_intro) # We Remove all html symbols ("\'s, \n")
        intro = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", intro_not_clean) # Remove any notes (in Wikipedia it links to another page)

        plot = re.sub('(?<=\w|\.|\s)((\\\'s)|(\\n){,10}|(\\\'))', "", raw_plot) # We all Remove html symbols ("\'s, \n")

        # Let's clean the infobox:

        info_dict = {}

        for key, value in raw_info.items():

            if key == "Title":          
                
                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value                

            elif key == "Directed by":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1])
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                    else:
                        info_dict[key] = value    
                else:
                    info_dict[key] = value

            elif key == "Produced by":
                
                if value != "NA":                    
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1])
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)   
                        
                        if re.search(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean):
                            split = re.split(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean) # Divide joint name (ex. Radish CamuJohn Snow)
                            info_dict[key] = ", ".join([i for i in split if i != ""]) # transform the split list a string (don't add empty stings)
                        else:
                            info_dict[key] = clean
                    else:

                        info_dict[key] = value                
                else:
                    
                    info_dict[key] = value
                    
            elif key == "Written by":

                if value != "NA":
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1])
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                        
                        if re.search(r'(?<=\w)\((.*?)\)', clean):
                            split = re.sub('(?<=\w)\((.*?)\)', "", clean) # Remove the parentesis with the role of each person
                            info_dict[key] = split
                        elif re.search(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean):
                            split = re.split(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean) # Divide joint name (ex. Radish CamuJohn Snow)
                            info_dict[key] = ", ".join([i for i in split if i != ""]) # transform the split list a string (don't add empty stings)
                        else: 
                            info_dict[key] = value
                    else: 
                        info_dict[key] = value
                else: 
                    info_dict[key] = value
                            

            elif key == "Starring":

                if value != "NA":                    
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1])
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                        
                        if re.search(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean):
                            split = re.split(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean) # Divide joint name (ex. Radish CamuJohn Snow)
                            info_dict[key] = ", ".join([i for i in split if i != ""]) # transform the split list a string (don't add empty stings)
                        else:
                            info_dict[key] = value
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value

            elif key == "Music by":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value

            elif key == "Release date":

                if value != "NA":                    
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1])
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean

                        if re.search(r'(\(\d+\))', clean): 
                            year = re.sub('(\(\d+\))', "", clean) # Get only the year (ex. 2005(2005))
                            info_dict[key] = year
                        elif re.search(r'(?<=\()(\d{4})', clean):
                            year = re.search('(?<=\()(\d{4})', clean) # Get only the year when (random characters(yyyy-mm-dd))
                            info_dict[key] = year.group()
                        else:
                            info_dict[key] = value
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value               

            elif key == "Running time":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value

            elif key == "Country":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean                       
                        
                        if re.search(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean):
                            split = re.split(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean) # Divide joint name (ex. Radish CamuJohn Snow)
                            info_dict[key] = ", ".join([i for i in split if i != ""]) # transform the split list a string (don't add empty stings)
                        else:
                            info_dict[key] = clean
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value

            elif key == "Language":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                        
                        if re.search(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean):
                            split = re.split(r'(?<!\s|\-|\_|\/)(?=[A-Z])', clean) # Divide joint name (ex. Radish CamuJohn Snow)
                            info_dict[key] = ", ".join([i for i in split if i != ""]) # transform the split list a string (don't add empty stings)
                        else:
                            info_dict[key] = clean       
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value

            elif key == "Budget":

                if value != "NA": 
                    if re.search(r'(?<=\w|\.|\s)(\[\d+\]){,10}', value): # Delet any reference (ex. [1]), We'll do for each variable and put always as first if
                        clean = re.sub(r'(?<=\w|\.|\s)(\[\d+\]){,10}', "", value)
                        info_dict[key] = clean
                    else:
                        info_dict[key] = value
                else:
                    info_dict[key] = value
        
    return [intro, plot, info_dict]

In [10]:
# Let's find all the required element for each movie:

## Guide to code:

The first three rows are the path for where the .html file have been saved, where we want to save the .tsv file and where we want to save the log file.

"i" represents the number of the file (it must be set depending on where we want to start), it must important to underline that it will be used to give a name to the saved file (and not to load data).

In the loop we load each .html file, open with beautifulsoup and apply the function developed above to get the information required to compelte the task and write them to different .tsv files (one for each .html file). We have got the title from the infobox because we have seen that it cleaner than the one get from teh tag "h1". We want the title in first position, then we get it from the dictionary and put as first element in the list that will be write on the file ("elements = [info_box[0]] + intro_plot + info_box[1:]"). 

As we said our code cannot generally work, some errors may be occur, that's why we created log file in which we print each element that is loaded by the function, in this way we know that if the program stops before the end, the last element is the that gives us problem. So print it and the error given, in this way it is easier to do the debug.

In [14]:
def save_as_tsv():

    path = "/Users/Dario/Desktop/HMW3_Data/" # Where the .html file have been saved
    save_in = "/Users/Dario/Desktop/HMW3_tsv/" # Where you want to save the .tsv file
    log_path = "/Users/Dario/Desktop/log.txt" # Where you want to save the log file

    log = open(log_path, "w")# A log file:  In case the some error occur we can see at which point it ccured
    
    i = 10000 # Change with 0 if you are working with all 30000 files or with the starting number of your file
    for file in sorted(os.listdir(path)):

        if file.startswith('article_'): # there is a hidden file whith another name, in this way we avoid to open it

            site = BeautifulSoup(open("".join([path, str(file)])), "html.parser") # Let's open each page
            
            log.write("".join([path, str(file)]) + "\n")

            try:
                with open("".join([save_in, "article_", str(i), ".tsv"]), "w") as file:
                    tsv_output = csv.writer(file, delimiter = '\t')
                    clean_el = clean_all(site) # Let's call the function that clean our data from "impurities"
                    intro_plot = [clean_el[0], clean_el[1]] 
                    info_box = [info for _, info in clean_el[2].items()]
                    elements = [info_box[0]] + intro_plot + info_box[1:] # Add as first element the title
                    tsv_output.writerow(elements)

            except Exception: # Let's see what is went wrong and break the cycle
                log.close()
                with open(log_path, "r") as log:
                    lines = log.read().splitlines() # Read only the last line of the log (it contains the article with the error)
                    last_line = lines[-1]
                    print("The error is in: ", last_line, traceback.print_exc())
                break

            i += 1
            
    log.close()
    print("Finish!")

In [15]:
save_as_tsv()

Finish!
